In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
#import train as train

import loss as loss
import unet as unet
import random
import paths as paths
import numpy as np
import glob

# The meaning of life
random.seed(42)  
np.random.seed(42)

In [ ]:
DATA_MEAN = 168.3172158554484
DATA_STD = 340.21625683608994
BATCH_SIZE = 1
OUTPUT_CHANNELS = 1
DATA_PATH = "/home/matthew/masters_code/dataset_vet_cleaned/"

In [ ]:
#WEIGHTS = "/home/matthew/masters_code/vacbag_dsc/dsc_after_bce_epoch_" + "80" + ".hdf5"
#WEIGHTS = "/home/matthew/masters_code/vacbag_tversky/tversky_epoch_" + "34" + ".hdf5"
WEIGHTS = "/home/matthew/masters_code/vacbag_bce/bce_epoch_60.hdf5"
INITIAL_LR = 1e-6
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = loss.dsc_loss
METRICS = [loss.dice_metric, loss.dsc, 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall()]

model = unet.model(output_channels=OUTPUT_CHANNELS)
model.compile(OPTIMIZER, LOSS, METRICS)
model.load_weights(WEIGHTS)

In [ ]:
patient_paths = paths.get_patient_paths(DATA_PATH)
patient_paths.sort()

img_paths = [glob.glob(path + "/img/*") for path in patient_paths]
mask_paths = [glob.glob(path + "/mask/*") for path in patient_paths]

valid = int(len(img_paths) * 0.15 // 1)
test = int(len(img_paths) * 0.1 // 1)
train = int(len(img_paths) - valid - test)

train_inputs = paths.flatten_list(img_paths[0:train])
train_truths = paths.flatten_list(mask_paths[0:train])

train_inputs.sort()
train_truths.sort()

valid_inputs = paths.flatten_list(img_paths[train:train+valid])
valid_truths = paths.flatten_list(mask_paths[train:train+valid])

valid_inputs.sort()
valid_truths.sort()

test_inputs = paths.flatten_list(img_paths[train+valid:])
test_truths = paths.flatten_list(mask_paths[train+valid:])

test_inputs.sort()
test_truths.sort()

In [ ]:
test_inputs = np.array([np.load(array) for array in test_inputs])
test_truths = np.array([np.load(array) for array in test_truths])

In [ ]:
all_truths = paths.flatten_list(mask_paths)
all_truths = np.array([np.load(array) for array in all_truths])
print(all_truths.shape)
img_size = 512*512
images = all_truths.shape[0]

In [ ]:
def perc_images(outputs, organ_index):
    images = outputs.shape[0]
    occurs = 0
    for x in outputs[...,organ_index]:
        if x.any() == 1:
            occurs += 1
    return (occurs / images) * 100

def mask_image_perc(outputs, organ_index):
    pixels = 0
    occurs = 0
    for x in outputs[...,organ_index]:
        if x.any() == 1:
            occurs += 1
            pixels += np.sum(x==1)
    return (pixels / (occurs * 512 * 512))*100

def mask_output_perc(outputs, organ_index):
    return mask_image_perc(outputs, organ_index) / outputs.shape[-1]

def pixels_total_perc(outputs, organ_index):
    pixels = 0
    for x in outputs[...,organ_index]:
        pixels += np.sum(x==1)
    return (pixels / np.prod(outputs.shape))*100

In [ ]:
print(perc_images(all_truths, 0))
print(mask_image_perc(all_truths, 0))
print(mask_output_perc(all_truths, 0))
print(pixels_total_perc(all_truths, 0))

In [ ]:
test_inputs = (test_inputs - DATA_MEAN) / DATA_STD

In [ ]:
test_inputs.shape, test_truths.shape

In [ ]:
# valid_inputs = np.array([np.load(array) for array in valid_inputs])
# valid_truths = np.array([np.load(array) for array in valid_truths])
# valid_inputs = (valid_inputs - DATA_MEAN) / DATA_STD

In [ ]:
# train_inputs = np.array([np.load(array) for array in train_inputs])
# train_truths = np.array([np.load(array) for array in train_truths])
# train_inputs = (train_inputs - DATA_MEAN) / DATA_STD

In [ ]:
model.evaluate(x=test_inputs, y=test_truths, batch_size=3)

In [ ]:
from matplotlib import pyplot as plt

#random_indexes = np.random.randint(0, test_inputs.shape[0], number)
random_indexes = [107, 120, 61, 95, 185]
number = len(random_indexes)
print(random_indexes)
fig, axs = plt.subplots(nrows=number, ncols=4, sharex=True, sharey=True, squeeze=True, figsize=(12,15))
for j, index in enumerate(random_indexes):
    ax = axs[j]
    
    for a in ax:
        a.set_xticklabels([])
        a.set_yticklabels([])
        a.set_xticks([])
        a.set_yticks([])
        
    img = test_inputs[index:index+1]
    truth = test_truths[index:index+1]
    pred = model.predict(img)
    pred = np.round(pred)
    diff = truth - pred
    
    ax[0].imshow(np.log(1+img[0,...,0]))
    ax[1].imshow(truth[0,...,0])
    ax[2].imshow(pred[0,...,0])
    ax[3].imshow(diff[0,...,0])
plt.tight_layout()

axs[0][0].set_title('Input', fontdict={'fontsize': 15, 'fontweight': 'medium'})
axs[0][1].set_title('Truth', fontdict={'fontsize': 15, 'fontweight': 'medium'})
axs[0][2].set_title('Output', fontdict={'fontsize': 15, 'fontweight': 'medium'})
axs[0][3].set_title('Difference', fontdict={'fontsize': 15, 'fontweight': 'medium'})

#plt.savefig("vacbag.svg", bbox_inches='tight')

In [ ]:
spacing_mm=[0.85, 0.85, 1.907]

from metrics import *

def google_metrics(y_true, y_pred, spacing_mm, organ_tol_mm, organ_index=None):
    if organ_index == None:
        y_pred = np.array(y_pred, dtype=bool)
        y_true = np.array(y_true, dtype=bool)
    else:
        y_pred = np.array(y_pred[...,organ_index], dtype=bool)
        y_true = np.array(y_true[...,organ_index], dtype=bool)
    surface_dice_list = []
    msd_list = []
    dice_list = []

    for true, pred in zip(y_true, y_pred):

        
        if np.max(true) >= 1 and np.max(pred) >= 1:
            surface_distances = compute_surface_distances(pred, true, spacing_mm)   
            surface_dice = compute_surface_dice_at_tolerance(surface_distances, organ_tol_mm)
            surface_dice_list.append(surface_dice)
            msd = np.mean(np.absolute(compute_average_surface_distance(surface_distances)))
            msd_list.append(msd)
            
            dice = compute_dice_coefficient(true, pred)
            dice_list.append(dice)
            
            
        elif np.max(true) >= 1 and np.max(pred) <= 1:
            msd_list.append(9999)
            surface_dice_list.append(0.00)
            dice_list.append(0.00)
            
        elif np.max(true) <= 1 and np.max(pred) <= 1:
            msd_list.append(0.00)
            surface_dice_list.append(1.00)          
            dice_list.append(1.00)
            
            
    return dice_list, surface_dice_list, msd_list

In [ ]:
from matplotlib import pyplot as plt

# number=5
# random_indexes = np.random.randint(0, test_inputs.shape[0], number)
# print(random_indexes)


organ_name = "vacbag"
organ_num = 0
random_indexes = [107, 120, 61, 95, 185]
number = len(random_indexes)
tol = 0.0
l=5





fig, axs = plt.subplots(nrows=number, ncols=4, sharex=True, sharey=True,squeeze=True, figsize=(12,12))
for j, index in enumerate(random_indexes):
    ax = axs[j]
    
    for a in ax:
        a.set_xticklabels([])
        a.set_yticklabels([])
        a.set_xticks([])
        a.set_yticks([])
        
    img = test_inputs[index:index+1,...]
    truth = test_truths[index:index+1,...,organ_num:organ_num+1]
    pred = model.predict(img)
    pred = np.round(pred[...,organ_num:organ_num+1])
    diff = truth - pred
    
    if np.max(truth) >= 1 and np.max(pred) >= 1:
        stats = google_metrics(truth, pred, spacing_mm, tol)
        print(stats)   
        stats = "DSC: {0:.3f} \nMSD: {1:.3f}".format(stats[0][0], stats[2][0])
        ax[3].text(1.05, 0.4, stats, size=18, ha="left", transform=ax[3].transAxes)

    
    ax[0].imshow(np.log(1+img[0,...,0][l:-l, l:-l]), cmap='gray')
    ax[0].contour(truth[0,...,0][l:-l, l:-l], colors='yellow', alpha=.25)
    ax[0].contour(pred[0,...,0][l:-l, l:-l],colors='red', alpha=.25)
    ax[1].imshow(truth[0,...,0][l:-l, l:-l])
    ax[2].imshow(pred[0,...,0][l:-l, l:-l])
    ax[3].imshow(diff[0,...,0][l:-l, l:-l])



axs[0][0].set_title('Input', fontdict={'fontsize': 23, 'fontweight': 'medium'})
axs[0][1].set_title('Truth (Y)', fontdict={'fontsize': 23, 'fontweight': 'medium'})
axs[0][2].set_title('Prediction (R)', fontdict={'fontsize': 23, 'fontweight': 'medium'})
axs[0][3].set_title('Difference', fontdict={'fontsize': 23, 'fontweight': 'medium'})

axs[0][0].text(-0.1,0.5, "A", size=23, ha="center", transform=axs[0][0].transAxes)
axs[1][0].text(-0.1,0.5, "B", size=23, ha="center", transform=axs[1][0].transAxes)
axs[2][0].text(-0.1,0.5, "C", size=23, ha="center", transform=axs[2][0].transAxes)
axs[3][0].text(-0.1,0.5, "D", size=23, ha="center", transform=axs[3][0].transAxes)
axs[4][0].text(-0.1,0.5, "E", size=23, ha="center", transform=axs[4][0].transAxes)
plt.tight_layout()
plt.savefig("/home/matthew/masters_presentation/images/vet_" + organ_name + ".png", bbox_inches='tight')


In [ ]:
# from metrics import *

# def google_metrics(y_true, y_pred, spacing_mm, organ_tol_mm=0):
#     y_pred = np.array(y_pred, dtype=bool)
#     y_true = np.array(y_true, dtype=bool)
    
#     surface_dice_list = []
#     msd_list = []
#     hd95_list = []
#     dice_list = []

#     for batch_index in range(y_true.shape[0]):
#         dice = compute_dice_coefficient(y_true, y_pred)
#         dice_list.append(dice)
#         if np.max(y_true[batch_index])> 0 or np.max(y_pred[batch_index]) > 0:
#             surface_distances = compute_surface_distances(y_pred[batch_index], y_true[batch_index], spacing_mm)
            
#             surface_dice = compute_surface_dice_at_tolerance(surface_distances, organ_tol_mm)
#             surface_dice_list.append(surface_dice)
            
#         if np.max(y_true[batch_index]) > 0 and np.max(y_pred[batch_index]) > 0:
#             msd = compute_average_surface_distance(surface_distances)
#             msd_list.append(msd)
            
#             hd95 = compute_robust_hausdorff(surface_distances, 0.95)
#             hd95_list.append(hd95)
            
            
            
            
#     return surface_dice_list, msd_list, hd95_list, dice_list

In [ ]:
preds = []
for index in range(len(test_inputs)):
    img = test_inputs[index:index+1]
    pred = model.predict(img)
    preds.append(pred[0])
    
preds = np.array(preds)
preds = np.round(preds)

In [ ]:
stats_patient = google_metrics(test_truths, preds, spacing_mm=[0.85, 0.85, 1.907], organ_tol_mm= 0)

In [ ]:
np.mean(stats_patient[0]), np.mean(stats_patient[1]), np.mean(stats_patient[2])

In [ ]:
plt.figure()
plt.imshow(test_truths[0,...,-1])
plt.figure()
plt.imshow(preds[0,...,-1])

In [ ]:
import medpy.metric.binary

In [ ]:
print(medpy.metric.binary.dc(preds, test_truths))
print(medpy.metric.binary.sensitivity(preds, test_truths))
print(medpy.metric.binary.specificity(preds, test_truths))
